In [ ]:
import json
import networkx as nx
from collections import defaultdict
from networkx.algorithms.simple_paths import shortest_simple_paths

def normalize_node(lp):
    return f"{lp['city'].strip()}, {lp['country'].strip()}"

def get_coords(lp):
    return tuple(lp["coordinates"])

def compute_paths(
    json_file="codes/cable-and-landing-points.json",
    allowed_countries=None,
    blacklisted_countries=None,
    damaged_cables=None,
    source_nodes=None,
    destination_nodes=None,
    max_paths_per_pair=10,
    max_cables_allowed=6,
    output_file="codes/timepass.geojson"
):
    # Default sets if not provided
    if allowed_countries is None:
        allowed_countries = {
            "India", "Australia", "Sri Lanka", "Myanmar", "Thailand", "Cambodia",
            "Vietnam", "Malaysia", "Indonesia", "Singapore", "Papua New Guinea",
            "Brunei", "Timor-Leste", "Philippines", "Solomon Islands"
        }
    if blacklisted_countries is None:
        blacklisted_countries = set()
    if damaged_cables is None:
        damaged_cables = set()

    with open(json_file, "r") as f:
        cable_and_landing_points = json.load(f)

    G = nx.Graph()
    node_coords = {}
    node_country = {}
    allowed_nodes = set()
    india_nodes = set()
    australia_nodes = set()
    cable_to_nodes = defaultdict(set)

    for cable in cable_and_landing_points:
        cable_name = cable["cable_name"]
        if cable_name in damaged_cables:
            continue

        lps = cable["landing_points"]
        for i in range(len(lps)):
            lp_i = lps[i]
            node_i = normalize_node(lp_i)
            coord_i = get_coords(lp_i)
            country_i = lp_i["country"]

            node_coords[node_i] = coord_i
            node_country[node_i] = country_i
            cable_to_nodes[cable_name].add(node_i)

            if country_i in allowed_countries and country_i not in blacklisted_countries:
                allowed_nodes.add(node_i)

            if country_i == "India":
                india_nodes.add(node_i)
            elif country_i == "Australia":
                australia_nodes.add(node_i)

            for j in range(i + 1, len(lps)):
                lp_j = lps[j]
                node_j = normalize_node(lp_j)
                coord_j = get_coords(lp_j)
                country_j = lp_j["country"]

                node_coords[node_j] = coord_j
                node_country[node_j] = country_j
                cable_to_nodes[cable_name].add(node_j)

                if country_j in allowed_countries and country_j not in blacklisted_countries:
                    allowed_nodes.add(node_j)

                if country_j == "India":
                    india_nodes.add(node_j)
                elif country_j == "Australia":
                    australia_nodes.add(node_j)

                if (country_i == country_j) or \
                   (country_i in blacklisted_countries) or \
                   (country_j in blacklisted_countries):
                    continue

                if country_i in allowed_countries and country_j in allowed_countries:
                    G.add_edge(node_i, node_j, cable=cable_name)

    subG = G.subgraph(allowed_nodes)

    if source_nodes is None:
        source_nodes = {
            node for node in india_nodes
            if node in subG and any(node_country[neighbor] != "India" for neighbor in subG.neighbors(node))
        }
    else:
        source_nodes = set(source_nodes).intersection(subG.nodes)

    if destination_nodes is None:
        destination_nodes = australia_nodes.intersection(subG.nodes)
    else:
        destination_nodes = set(destination_nodes).intersection(subG.nodes)

    valid_paths = []
    for source in source_nodes:
        for target in destination_nodes:
            try:
                path_gen = shortest_simple_paths(subG, source, target)
                count = 0
                for path in path_gen:
                    if count >= max_paths_per_pair:
                        break

                    cables = []
                    valid = True
                    for i in range(len(path) - 1):
                        n1, n2 = path[i], path[i + 1]
                        edge_data = subG.get_edge_data(n1, n2)
                        cable = edge_data.get("cable")

                        if not cable or cable in damaged_cables:
                            valid = False
                            break
                        if node_country[n1] == node_country[n2]:
                            valid = False
                            break

                        cables.append(cable)

                    if valid and len(set(cables)) <= max_cables_allowed:
                        valid_paths.append({
                            "path": path,
                            "cables": cables
                        })
                        count += 1
            except nx.NetworkXNoPath:
                continue

    geojson_features = []
    for item in valid_paths:
        coordinates = [node_coords[n] for n in item["path"]]
        feature = {
            "type": "Feature",
            "properties": {
                "path": item["path"],
                "cables": item["cables"],
                "cable_count": len(set(item["cables"]))
            },
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            }
        }
        geojson_features.append(feature)

    geojson_output = {
        "type": "FeatureCollection",
        "features": geojson_features
    }

    with open(output_file, "w") as f:
        json.dump(geojson_output, f, indent=2)

    print(f"Saved {len(geojson_features)} valid paths using ≤ {max_cables_allowed} cables.\n")
    for item in valid_paths:
        print(" → ".join(item["path"]))
        print("  Cables:", " → ".join(item["cables"]))
        print()

# Example usage:
# compute_paths(blacklisted_countries={}, damaged_cables={})



def get_dropdown_options(json_file="codes/cable-and-landing-points.json"):
    with open(json_file, "r") as f:
        cable_and_landing_points = json.load(f)

    india_nodes = set()
    australia_nodes = set()
    all_countries = set()
    all_cables = set()

    for cable in cable_and_landing_points:
        all_cables.add(cable["cable_name"])
        for lp in cable["landing_points"]:
            country = lp["country"]
            city = lp["city"]
            node = f"{city.strip()}, {country.strip()}"

            all_countries.add(country)
            if country == "India":
                india_nodes.add(node)
            elif country == "Australia":
                australia_nodes.add(node)

    return {
        "source_nodes": sorted(india_nodes),
        "destination_nodes": sorted(australia_nodes),
        "countries": sorted(all_countries),
        "cables": sorted(all_cables),
    }


    

Available cables to disable:
 1. Apricot
 2. Asia Africa Europe-1 (AAE-1)
 3. Asia Connect Cable-1 (ACC-1)
 4. Asia-America Gateway (AAG) Cable System
 5. Australia-Singapore Cable (ASC)
 6. Batam Dumai Melaka (BDM)
 7. Bay of Bengal Gateway (BBG)
 8. Bharat Lanka Cable System
 9. Bifrost
 10. Coral Sea Cable System (CS²)
 11. FALCON
 12. Hawaiki Nui 1
 13. ICE IV
 14. INDIGO-West
 15. India Asia Xpress (IAX)
 16. Indonesia Global Gateway (IGG) System
 17. JAKABARE
 18. Jakarta-Bangka-Bintan-Batam-Singapore (B3JS)
 19. MIST
 20. SAFE
 21. SEA-H2X
 22. SeaMeWe-4
 23. SeaMeWe-5
 24. Tata TGN-Tata Indicom
 25. Thailand-Indonesia-Singapore (TIS)


Removed edges for cable 'Tata TGN-Tata Indicom' (total removed: 1)


Finding alternate routes from Chennai, India to Brisbane, Australia...

Route 1:
  Path:   Chennai, India -> Jakarta, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 2:
  Path:   Chennai, India -> Singapore, Singapore -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 3:
  Path:   Chennai, India -> Ancol, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 4:
  Path:   Chennai, India -> Jakarta, Indonesia -> Singapore, Singapore -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 -> Hawaiki Nui 1 

Route 5:
  Path:   Chennai, India -> Singapore, Singapore -> Jakarta, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 -> Hawaiki Nui 1 

